# <span style="color:blue"> Projeto 01 - Concessão de cartões de crédito </span>
<span style="color:blue"> Este notebook é semelhante ao visto em vídeo, mas contém células azuis como esta, que trazem instruções para a sua atividade.</span>

<span style="color:blue">Após realizar as tarefas indicadas, você vai fazer o upload do seu arquivo no GitHub e enviar o link para a EBAC, ou alternativamente, fazer o upload do arquivo na plataforma da EBAC. Recomendamos o github, pois assim você já vai montando o seu portfólio.</span>

## Etapa 1 CRISP - DM: Entendimento do negócio

Como primeira etapa do CRISP-DM, vamos entender do que se trata o negócio, e quais os objetivos. 

Este é um problema de concessão de cartões de crédito, publicado no [Kaggle](https://www.kaggle.com/), uma plataforma que promove desafios de ciência de dados, oferecendo prêmios em dinheiro para os melhores colocados. O link original está [aqui](https://www.kaggle.com/rikdifos/credit-card-approval-prediction).  
  
Essa é uma base de proponentes de cartão de crédito, nosso objetivo é construir um modelo preditivo para identificar o risco de inadimplência (tipicamente definida pela ocorrência de um atraso maior ou igual a 90 em um horizonte de 12 meses) através de variáveis que podem ser observadas na data da avaliação do crédito (tipicamente quando o cliente solicita o cartão).

Atividades do CRISP-DM:

- Objetivos do negócio
Note que o objetivo aqui é que o modelo sirva o mutuário (o cliente) para que avalie suas próprias decisões, e não a instituição de crédito.
- Objetivos da modelagem
O objetivo está bem definido: desenvolver o melhor modelo preditivo de modo a auxiliar o mutuário a tomar suas próprias decisões referentes a crédito.
  
Nessa etapa também se avalia a situação da empresa/segmento/assunto de modo a se entender o tamanho do público, relevância, problemas presentes e todos os detalhes do processo gerador do fenômeno em questão, e portanto dos dados.

Também é nessa etapa que se constrói um planejamento do projeto.

## Etapa 2 Crisp-DM: Entendimento dos dados
A segunda etapa é o entendimento dos dados. Foram fornecidas 15 variáveis mais a variável resposta (em negrito na tabela). O significado de cada uma dessas variáveis se encontra na tabela.

#### Dicionário de dados

Os dados estão dispostos em uma tabela com uma linha para cada cliente, e uma coluna para cada variável armazenando as características desses clientes. Colocamos uma cópia o dicionário de dados (explicação dessas variáveis) abaixo neste notebook:

| Variable Name            | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| sexo| M = 'Masculino'; F = 'Feminino' |M/F|
| posse_de_veiculo| Y = 'possui'; N = 'não possui' |Y/N|
| posse_de_imovel| Y = 'possui'; N = 'não possui' |Y/N|
| qtd_filhos| Quantidade de filhos |inteiro|
| tipo_renda|Tipo de renda (ex: assaliariado, autônomo etc) | texto |
| educacao| Nível de educação (ex: secundário, superior etc) |texto|
| estado_civil | Estado civil (ex: solteiro, casado etc)| texto |
| tipo_residencia | tipo de residência (ex: casa/apartamento, com os pais etc) | texto |
| idade | idade em anos |inteiro|
| tempo de emprego | tempo de emprego em anos |inteiro|
| possui_celular | Indica se possui celular (1 = sim, 0 = não) |binária|
| possui_fone_comercial | Indica se possui telefone comercial (1 = sim, 0 = não) |binária|
| possui_fone | Indica se possui telefone (1 = sim, 0 = não) |binária|
| possui_email | Indica se possui e-mail (1 = sim, 0 = não) |binária|
| qt_pessoas_residencia | quantidade de pessoas na residência |inteiro|
| **mau** | indicadora de mau pagador (True = mau, False = bom) |binária|





#### Carregando os pacotes
É considerado uma boa prática carregar os pacotes que serão utilizados como a primeira coisa do programa.

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

#### Carregando os dados
O comando pd.read_csv é um comando da biblioteca pandas (pd.) e carrega os dados do arquivo csv indicado para um objeto *dataframe* do pandas.

In [ ]:
# Observe que demo01.csv está na mesma pasta que este notebook
# do contrário, seria necessário indicar a pasta no nome do arquivo
df = pd.read_csv('demo01.csv')
print ("Número de linhas e colunas da tabela: {}".format(df.shape))

df.head()

#### Entendimento dos dados - Univariada
Nesta etapa tipicamente avaliamos a distribuição de todas as variáveis. Nesta demonstração vamos ver a variável resposta e dois exemplos de univariada apenas. Mas sinta-se à vontade para tentar observar outras variáveis.

In [ ]:
print(df['mau'].value_counts())
print("\nTaxa de inadimplentes:")
print(df['mau'].mean())

In [ ]:
var = 'mau'
grafico_barras = df[var].value_counts().plot.bar()

### <span style="color:blue">Tarefa 01 - gráfico de barras</span>
<span style="color:blue"> Com base no código da célula anterior, construa um gráfico de barras para pelo menos duas outras variáveis. 
**Dica:** Não tente usar as variáveis ```tempo_emprego``` e ```idade``` pois o gráfico de barras dessa forma como construímos não é adequado para elas. </span>

In [ ]:
def calc_freq_abs(df: pd.DataFrame, col_calc: str, col_ref:str = None, val_ref = None) -> list:
    aux = []
    if val_ref == None:
        aux = df[col_calc].value_counts()
    else:
        aux = df[df[col_ref]==val_ref][col_calc].value_counts()
    return aux

def calc_freq_rel(df: pd.DataFrame, col_calc: str, col_ref:str = None, val_ref = None) -> list:
    return calc_freq_abs(df,col_calc,col_ref,val_ref)/df.shape[0]


def break_apart(l: list = [], df: pd.DataFrame = pd.DataFrame(), col: str = None):
    list_aux = []
    if not bool(l) and col:
        l = list(df[col])
    for i in l:
        if i not in list_aux:
            list_aux.append(i)
    return list_aux

def df_abs_freq(l:list) -> pd.DataFrame:
    df1 = pd.DataFrame(data=l.index, columns=['tipo'])
    df2 = pd.DataFrame(list(l), columns=['freq_abs'])
    df3 = pd.DataFrame.merge(df1, df2, left_index=True, right_index=True)
    return df3


In [ ]:
# Bloco de codigo [6] criando graficos de colunas com valores absolutos para as
# variaveis categoricas do lista list_cols.
# Cada variavel ganha 3 graficos, 1º com o total, 2º com os bons pagadores, 3º com os maus pagadores
x=0
y=0
cont=0
list_cols = ['tipo_renda', 'educacao', 'estado_civil']
col_r = 'mau'
lx = (None, False, True)

aux_lx=[]
temp_x =[]
aux_ly = break_apart(df=df, col=list_cols)

figura, ex = plt.subplots(len(aux_ly),len(lx), figsize=(25, 12))

for i in aux_ly:
  for j in lx:
    temp_lx = calc_freq_abs(df=df,col_calc=i,col_ref='mau',val_ref=j)
    aux_lx.append(df_abs_freq(temp_lx))

    x= cont%3
    y= int(np.trunc(cont/3))
    f = sns.barplot(x='tipo', y='freq_abs', data=aux_lx[cont], ax=ex[y,x])
    if lx[x]==None:
      n_x = 'TOTAL'
    else:
      n_x = f'Mau Pagador? {lx[x]}'
    f.set(title=n_x, xlabel='', ylabel='Frequência Absoluta')
    plt.setp(f.get_xticklabels(), rotation=30)
    cont += 1

plt.subplots_adjust(bottom=0.01, top=1.5, wspace=1.0, hspace=1.0)

figura.show()

In [ ]:
# Bloco de codigo [7] mostrando a relação de Maus pagadores por valor de cada variavel categorica
# existente na lista de nome list_cols
# As relações foram transformadas em pd.DataFrame para serem melhores visualizadas
list_rel = []
for i in list_cols:
  df_rel = df_abs_freq(calc_freq_rel(df, i, 'mau', True))
  list_rel.append(df_rel)

print(list_cols[0])
print(list_rel[0])
print()
print(list_cols[1])
print(list_rel[1])
print()
print(list_cols[2])
print(list_rel[2])


In [ ]:
plt.clf()
var = "tempo_emprego"

sns.displot(df, x = var, bins = 50)
plt.show()

In [ ]:
# Alterando valores de -1000 pra -2, para visualizar melhor no gráfico
var = "tempo_emprego"
df.loc[df[var]<0,var] = -2

In [ ]:
plt.clf()
var = "tempo_emprego"

sns.displot(df, x = var, bins = 50)
plt.show()

### <span style="color:blue">Tarefa 02 - Histograma </span>
<span style="color:blue"> Com base no código da célula anterior, construa o histograma da variavel ```dade```.  </span>


In [ ]:
# Bloco de codigo [11] construção de 3 graficos historigrama para a variavel quantitativa continua
# "idade", sendo o 1º com todos os valores dessa variavel, ela representa quase uma normal, o 2º
# nos mostra os Maus Pagadores, a uma predominancia de maus pagadores nos estremos, criando
# um grafico em forma de "U", já o 3º grafico nos dá novamente algo parecido com uma normal 

lx = (None, False, True)
plt.clf()
var = "idade"

figura, ax = plt.subplots(3,0, figsize=(25, 12))
ax[0]
f1 = sns.displot(df, x = var, bins = 50)
f1.set(title='TODOS', xlabel='idade')
ax[1]
f2 = sns.displot(df[df['mau']==True], x = var, bins = 50)
f2.set(title='MAU PAGADOR', xlabel='idade')
ax[2]
f3 = sns.displot(df[df['mau']==False], x = var, bins = 50)
f3.set(title='BOM PAGADOR', xlabel='idade')
plt.show()

### Entendimento dos dados - Bivariadas

Entender a alteração da inadimplência indicada pela variável resposta (```AtrasoRelevante2anos```) e as variáveis explicativas (demais). Para isto, vamos calcular a taxa de inadimplentes (qtd inadimplentes / total) para diferentes grupos definidos pelas variáveis explicativas.


In [ ]:
var = 'idade'
cat_srs, bins = pd.qcut(df[var], 4, retbins=True)
g = df.groupby(cat_srs)
biv = g['mau'].mean()

ax = biv.plot.line()
ax.set_ylabel("Proporção de inadimplentes")
ticks = plt.xticks(range(len(biv.index.values)), biv.index.values, rotation = 90)

### <span style="color:blue">Tarefa 03 - Bivariada </span>
<span style="color:blue"> Com base no código da célula anterior, construa uma análise bivariada para a variável  ```tempo_emprego```.  Em seguida, insira uma célula de markdown e conclua se a variável parece discriminar risco de crédito. </span>



In [ ]:
var = "tempo_emprego"
cat_srs, bins = pd.qcut(df[var], 4, retbins=True)
g = df.groupby(cat_srs)
biv = g['mau'].mean()

ax = biv.plot.line()
ax.set_ylabel("Proporção de inadimplentes")
ticks = plt.xticks(range(len(biv.index.values)), biv.index.values, rotation = 90)

**1. ANALISE**

1.1. VARIAVEIS CATEGORICAS

  Foi contruido 3 graficos de colunas para cada uma das seguintes variaveis categoricas: tipo_renda, educacao, estado_civil. O 1º Grafico, representa o total de individos de cada categoria, o 2º representa quanto desse total responde a seguinte pergunta, "Mau Pagador?", como False, ja o 3º representa o quanto desse total responde a seguinte pergunta, "Mau Pagador?", como True.

  Verificando os graficos e comparando, não existe nenhum padrão de relacionamento que seja muito forte.

  Por tanto foi contruido em sequncia pd.DataFrames que possuem a frequencia relatica, dessas variaveis com o estatos de "Mau Pagador". Essa tabela nos permitiu chegar a algumas conclusões:
  
  1.1.1. **Tipo de Renda** 
  
  Analizando esse item notamos que os maiores devedores ganham a vida como "Working", tendo uma inadiplencia de 1,13% o dobro da segunda modalidade de renda, "Commercial associate" com um grau de individamento em 0,59%.

  1.1.2. **Educacao**

  O grande devedor nesse caso são aqueles q possuem formação academica como: 0  "Secondary / secondary special", tendo um grão de individamento na ordem de 1,46% contra: "Higher education" com 0,69%, "Incomplete higher"  0,15% e "Lower secondary" com 0,04%

  1.1.3. **Estado Civil**

  Verificando o estado civil, percebe-se que os mais indicidados são os que estão casados, veja: "Married" tem 1,45%, todos os outros tem valores abaixo de 0,4%.

1.2 VARIAVEIS QUANTITATIVAS CONTINUAS

1.2.1. **IDADE**

Foram criados 3 historigramas, sendo o 1º com o valor total das idades, esse possui uma forma proxima a Normal, o 2º com as idades apenas dos "Maus Pagadores", esse ja possui uma forma mais proxima a de um "U", e por fim o grafico do "Bons Pagadores" que novamente se aproxima de uma Normal. 
Tanto o historigrama quando o poligono de frequncia, nos permitem entender que a faixa com maior inadiplencia entá entre os mais jovens, essa inadiplencia cai rapidamente, tendo seu menor ponto por volta dos 35 anos, depois volta a crescer com um angulo de inclinação bem mais suave.

1.2.2. ** Tempo de Emprego **

Atraves do grafico de poligono de frequencia, exibe um auto individamente entre os desempregados e os com pouco tempo de emprego, tendo seu apce em 1,2 anos de trabalho e depois caindo com um coeficiente linear bem elevado até os 4,7 anos de trabalho, depois continua caindo mas de forma mais amena.

**2. CONCLUSÃO**

Recomendamos realizar os emprestimos para individuos adultos com cerca de 3 decadas de vida, e com o maior tempo de emprego possivel, as variaveis categoricas tem uma variação entre elas pouco relevantes para uma analise grafica, assim em primeira analise podemos descartalas como metricas de decisão.


## Etapa 3 Crisp-DM: Preparação dos dados
Nessa etapa realizamos tipicamente as seguintes operações com os dados:
- seleção
Neste caso, os dados já estão pré-selecionados
- limpeza
Precisaremos identificar e tratar dados faltantes
- construção
Neste primeiro exercício não faremos construção de novas variáveis
- integração
Temos apenas uma fonte de dados, não é necessário agregação
- formatação
Os dados já se encontram em formatos úteis

Os dados já estão pré-selecionados, construídos e integrados, mas há dados faltantes que serão eliminados na próxima célula

In [ ]:
metadata = pd.DataFrame(df.dtypes, columns = ['tipo'])

metadata['n_categorias'] = 0

for var in metadata.index:
    metadata.loc[var,'n_categorias'] = len(df.groupby([var]).size())
    
metadata

In [ ]:
def convert_dummy(df, feature,rank=0):
    pos = pd.get_dummies(df[feature], prefix=feature)
    mode = df[feature].value_counts().index[rank]
    biggest = feature + '_' + str(mode)
    pos.drop([biggest],axis=1,inplace=True)
    df.drop([feature],axis=1,inplace=True)
    df=df.join(pos)
    return df

In [ ]:
for var in metadata[metadata['tipo'] == 'object'].index:
    df = convert_dummy(df, var)

In [ ]:
df

## Etapa 4 Crisp-DM: Modelagem
Nessa etapa que realizaremos a construção do modelo. Os passos típicos são:
- Selecionar a técnica de modelagem
Utilizaremos a técnica de floresta aleatória (**random forest**), pois é uma técnica bastante versátil e robusta que captura bem padrões complexos nos dados, relativamente fácil de se usar e que costuma produzir excelentes resultados para uma classificação como estas. Vamos ver esse algoritmo em detalhes mais adiante no curso, mas pense nele por enquanto como uma regra complexa baseada nas variáveis explicativas que classifica o indivíduo como inadimplente ou não. Mais adiante no curso vamos extrair mais dessa técnica.
- Desenho do teste
Antes de rodar o modelo precisamos construir um desenho do teste que será realizado. Para desenvolver um modelo como este, é considerado uma boa prática dividir a base em duas, uma chamada ```treinamento```, onde o algoritmo 'aprende', e outra chamada ```teste```, onde o algoritmo é avaliado. Essa prática fornece uma métrica de avaliação mais fidedigna do algoritmo, falaremos mais detalhes em lições futuras.
- Avaliação do modelo
Faremos a avaliação do nosso modelo através do percentual de acerto, avaliando a classificação do modelo (inadimplente e não inadimplente) e comparando com o estado real armazenado na variável resposta (```AtrasoRelevante2anos```). Esse percentual de acerto é frequentemente chamado de acurácia (**obs:** nunca usar assertividade... a**ss**ertivo não é aquele que a**c**erta, e sim "*adj.: em que o locutor declara algo, positivo ou negativo, do qual assume inteiramente a validade; declarativo*." a**C**ertivo está errado ;)
#### Dividindo a base em treino e teste



In [ ]:
# Tirando a v. resposta da base de treinamento
x = df.drop("mau",axis = 1)
y = df["mau"]

# Tirando ID da base de treinamento e teste
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [ ]:
x_train

### Rodando o modelo
A função RandomForestClassifier gera a estrutura da floresta aleatória, e o parâmetro ```n_estimator``` define o número de árvores na floresta. Normalmente a acurácia do modelo tende a aumentar com o número de árvores, pelo menos até um certo limite - e aumenta também o recurso computacional demandado. Você pode alterar esse parâmetro e verificar se a acurácia do seu modelo melhora - não recomendamos valores muito altos. Vá alterando aos poucos e percebendo como o tempo aumenta com os seus recursos. Não é necessário ir muito além de umas 100 árvores.

In [ ]:
# Treinar uma Random Forest com 5 árvores

clf = RandomForestClassifier(n_estimators=3)
clf.fit(x_train,y_train)

In [ ]:
# Calculando a acuracia

y_pred = clf.predict(x_test)
acc = metrics.accuracy_score(y_test, y_pred)
print('Acurácia: {0:.2f}%'.format(acc*100))

In [ ]:
# Matriz de confusão

tab = pd.crosstab(index = y_pred, columns = y_test)
print(tab[1][0]/(tab[1][0] + tab[0][0]))
print(tab[1][1]/(tab[1][1] + tab[0][1]))
tab

### <span style="color:blue">Tarefa 04 - Bivariada </span>
<span style="color:blue"> Para essa tarefa, crie três células extras, copie nelas o código das três células anteriores (na mesma ordem) e altere o parâmetro ```n_estimators=``` da função ```RandomForestClassifier``` e insira uma quantidade maior que 3 nesse parâmetro. Rode as três células anteriores para calcular a acurácia do modelo e veja se você consegue uma acurácia melhor. </span>

In [ ]:
# Treinar uma Random Forest com 5 árvores
clf = RandomForestClassifier(n_estimators=80)
clf.fit(x_train,y_train)

# Calculando a acuracia
y_pred = clf.predict(x_test)
acc = metrics.accuracy_score(y_test, y_pred)
print('Acurácia: {0:.2f}%'.format(acc*100))
print()

# Matriz de confusão
tab = pd.crosstab(index = y_pred, columns = y_test)
print(tab[1][0]/(tab[1][0] + tab[0][0]))
print(tab[1][1]/(tab[1][1] + tab[0][1]))
tab

In [ ]:
# Treinar uma Random Forest com 5 árvores

clf = RandomForestClassifier(n_estimators=10)
clf.fit(x_train,y_train)

In [ ]:
# Calculando a acuracia

y_pred = clf.predict(x_test)
acc = metrics.accuracy_score(y_test, y_pred)
print('Acurácia: {0:.2f}%'.format(acc*100))

In [ ]:
# Matriz de confusão

tab = pd.crosstab(index = y_pred, columns = y_test)
print(tab[1][0]/(tab[1][0] + tab[0][0]))
print(tab[1][1]/(tab[1][1] + tab[0][1]))
tab

## Etapa 5 Crisp-DM: Avaliação dos resultados
A etapa final do CRISP. Neste casp, a nossa avaliação termina com a acurácia. Mas em problemas futuros aprofundaremos mais - a ideia seria avaliar o impacto do uso do modelo no negócio, ou seja, o quanto o resultado financeiro melhora em detrimento da utilização do modelo.

Como um exemplo simples, considere que um cliente bom pagador deixa (em média) 5 '*dinheiros*' de lucro, e um mau pagador deixa (em média) 100 '*dinheiros*' de prejuízo.

de acordo com a matriz de confusão:

| Decisão   | lucro dos bons    | lucro dos maus | total  |
| --------- |:-----------------:|:--------------:| ------:|
| Aprovador | 4042 x 5          | 72 x (-100)    | 13.010 |
| Reprovar  |  27 x 5           | 22 x (-100)    | -2.065 |

Estariamos evitando, portanto, um prejuízo de -2.145 '*dinheiros*' - o que na prática significa um aumento no lucro.


## Etapa 6 Crisp-DM: Implantação
Nessa etapa colocamos em uso o modelo desenvolvido, normalmente implementando o modelo desenvolvido em um motor de crédito que toma as decisões com algum nível de automação - tipicamente aprovando automaticamente clientes muito bons, negando automaticamente clientes muito ruins, e enviando os intermediários para análise manual.